# Import library

In [151]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

# Read data and split to x labels

In [152]:
# path to the database - change it if needed
path = "./face_data/"
ids = range(1, 16) # 15 people
states = ['centerlight', 'glasses', 'happy', 'leftlight',
'noglasses', 'normal', 'rightlight','sad',
'sleepy', 'surprised', 'wink' ]

prefix = 'subject'
surfix = '.png' #file extension is png

# open one picture to get the image's size
fn = prefix + '01.' + states[0] + surfix
im = cv2.imread(path + fn, 0)

h = im.shape[0] # hight
w = im.shape[1] # width
D = h * w
N = len(states)*15
print(N, D, h, w)

X = np.zeros((D, N))
labels = [i for j in range(15) for i in range(11) ]
print(labels)
# collect all data
count = 0
# there are 15 people
for person_id in range(1, 16):
    for state in states:
        # get name of each image file
        fn = path + prefix + str(person_id).zfill(2) + '.' + state + surfix
        # open the file and read as grey image
        tmp = cv2.imread(fn, cv2.IMREAD_GRAYSCALE)
        # then add image to dataset X
        X[:, person_id] = tmp.reshape(D)
        count += 1
print(np.array(labels).shape)


165 77760 243 320
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
(165,)


# Standard and reduce dimensions

In [153]:
X = StandardScaler().fit_transform(X.T)
pca = PCA(n_components = 100)
X_transform = pca.fit_transform(X)
X_train, X_test, Y_train, Y_test = train_test_split(X_transform, labels, test_size=0.3, random_state=25)
print(X_train.shape)

(115, 100)


# Fitting with Multiple layer perceptron

In [154]:
clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(100), random_state=40)
clf.fit(X_train, Y_train)

MLPClassifier(alpha=1e-05, hidden_layer_sizes=100, random_state=40,
              solver='lbfgs')

# Predict with train data

In [155]:
y_pre = clf.predict(X_train)
print("accuracy of with train data", accuracy_score(y_pre, Y_train))

accuracy of with train data 0.21739130434782608


# Predict with test data

In [156]:
y_pre = clf.predict(X_test)
print("predict test data", y_pre)
print("accuracy of with test data", accuracy_score(y_pre, Y_test))

predict test data [7 7 7 7 7 7 7 7 7 7 7 7 7 7 4 7 7 7 7 7 7 7 8 7 7 7 7 7 7 7 7 7 7 7 7 7 7
 7 7 7 7 7 7 7 7 7 7 7 7 7]
accuracy of with test data 0.04


# Use multinomial logistic regression

In [157]:
lorg = LogisticRegression(multi_class='multinomial',solver='sag', max_iter=5000)
lorg.fit(X_train, Y_train)
y_pre = lorg.predict(X_test)

print(y_pre)
print("Accuracy score with test data", accuracy_score(y_pre, Y_test))

[7 7 7 7 7 7 7 7 7 7 7 7 7 7 3 7 7 7 7 7 7 7 3 7 7 7 7 7 7 7 7 7 7 7 7 7 7
 7 7 7 7 7 7 7 7 7 7 7 7 7]
Accuracy score with test data 0.04


# Read 5 image for test data

In [158]:
H = 320
W = 243
D = H * W
N = 5

X_test_image = np.zeros((D,5))
for i in range(0,5):
    path = "./test" + str(i + 1) + ".png"
    img = cv2.imread(path, 0)
    img.reshape(243, 320)
    X_test_image[:, i] = img.reshape(D)

transformed_test_img = X_test_image.T.dot(pca.components_.T)
print(transformed_test_img.T.shape)


(100, 5)


# Fitting with x is train data and show accuracy with train data

In [159]:
print(X_transform.shape)
clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(100), random_state=40)
clf.fit(X_transform, labels)
print(accuracy_score(labels, clf.predict(X_transform)))

(165, 100)
0.17575757575757575


# Predict test image

In [160]:
# Then Predict the Test data
Y_pred = clf.predict(transformed_test_img)
print([states[x] for x in Y_pred])

['sleepy', 'sleepy', 'sleepy', 'surprised', 'sleepy']
